## <font color="#dd0000">Notice</font>

1. You must use **GPU**.
2. You are a **Google Colab Pro** user.

Otherwise you will not be able to use colab!!!

## Clone repo

In [ ]:
!git clone https://github.com/Vision-CAIR/MiniGPT-4.git

Cloning into 'MiniGPT-4'...
remote: Enumerating objects: 241, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 241 (delta 66), reused 50 (delta 37), pack-reused 143
Receiving objects: 100% (241/241), 57.52 MiB | 36.74 MiB/s, done.
Resolving deltas: 100% (90/90), done.


# Check gpu

In [ ]:
!nvidia-smi

Thu Apr 20 17:50:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Install package

In [ ]:
ls

In [ ]:
%cd MiniGPT-4/

In [ ]:
!pip install -qr requirements.txt

In [ ]:
!pip install -q salesforce-lavis
!pip install -q bitsandbytes
!pip install -q accelerate
!pip install -q gradio==3.27.0

In [ ]:
!pip install -q h5py
!pip install -q typing-extensions
!pip install -q wheel
!pip install -q git+https://github.com/huggingface/transformers.git -U

# Download pretrained_minigpt4

In [ ]:
!wget https://huggingface.co/wangrongsheng/MiniGPT4/blob/main/pretrained_minigpt4.pth

--2023-04-20 17:51:47--  https://huggingface.co/wangrongsheng/MiniGPT4/blob/main/pretrained_minigpt4.pth
Resolving huggingface.co (huggingface.co)... 18.67.0.55, 18.67.0.67, 18.67.0.34, ...
Connecting to huggingface.co (huggingface.co)|18.67.0.55|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42713 (42K) [text/html]
Saving to: ‘pretrained_minigpt4.pth’

pretrained_minigpt4 100%[===================>]  41.71K  --.-KB/s    in 0.02s   

2023-04-20 17:51:47 (2.04 MB/s) - ‘pretrained_minigpt4.pth’ saved [42713/42713]



## <font color="#660000">Notice</font>

- Set `llama_model: "wangrongsheng/MiniGPT-4-LLaMA"` in `minigpt4/configs/models/minigpt4.yaml`
- Set `ckpt: 'pretrained_minigpt4.pth'` in `eval_configs/minigpt4_eval.yaml`

> `wangrongsheng/MiniGPT-4-LLaMA` was created from `vicuna-13b-delta-v0` and `llama-13b-hf`

# Run MiniGPT-4

In [ ]:
!python demo.py --cfg-path eval_configs/minigpt4_eval.yaml

/usr/local/lib/python3.9/dist-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
2023-04-20 18:11:18.964696: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Initializing Chat
Loading VIT
Loading VIT Done
Loading Q-Former
Loading Q-Former Done
Loading LLAMA

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
Loading checkpoint shards:   0% 0/3 [00:00<?, ?it/s]^C
